In [1]:
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from collections import defaultdict
import pyodbc 
import pandas as pd
import numpy as np
import umap
from sklearn import cluster

In [2]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DLRBIMSQL;'
                      'Database=ClarityIndex;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [60]:
query = """ SELECT  RoomId,RoomName
FROM [ClarityIndex].[dbo].[PQ_RoomContents] PQRC
group by RoomId,RoomName  """

In [61]:
RNS = pd.read_sql_query(query,conn)

In [63]:
import nltk
def tokenlenfilter(tokenlist,minlen=2):
    tokens = [x for x in tokenlist if len(x)>minlen]
    return (tokens)

def get_ngrams(series: pd.Series,n:int):
    ngrams = series.copy().str.split(' ').explode()
    ngrams1 = series.copy().str.split(' ').explode()
    for i in range(1,n):
        foo=ngrams1.groupby(level=0).shift(-i)
        ngrams=ngrams.str.cat(foo,sep=' ')
    ngrams = ngrams.dropna()
    return pd.DataFrame(ngrams)

In [64]:
RNS['Name2']=RNS['RoomName'].str.lower().str.replace('\d+','').str.replace('\+','').replace('[^\w\s]',' ').\
    str.lstrip().str.rstrip('-').str.rstrip()
RNS['Name2']=RNS['Name2'].str.replace('\.','').str.replace('\\',' ').str.replace('-',' ').str.replace('/',' ').str.replace("''",'')
uncommontokens=list(get_ngrams(RNS['Name2'],1)['Name2'].value_counts()[get_ngrams(RNS['Name2'],1)['Name2'].value_counts()<3].index.values)

C:\Users\dmckenzie\Anaconda3\envs\Three Seven\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

The default value of regex will change from True to False in a future version.



In [67]:
def tokennamefilter(tokenlist,minlen=2):
    tokens = [x for x in tokenlist if x not in uncommontokens+['rm','room','men','women','mens','inmate','female','male',
                                                               'ada','accessible',
                                                               'womens','girl','boy','boys','girls',
                                                              'east','west','north','south','space','area','zone',
                                                              'small','medium','large','xlarge']]
    return (tokens)


In [71]:
RNS['NameFilter']=RNS['Name2'].apply(nltk.word_tokenize).\
    apply(tokenlenfilter).\
    apply(tokennamefilter)
for i  in RNS['NameFilter']:
    if len(i)<3:
        i.sort()
RNS['NameFilter']=RNS['NameFilter'].apply(' '.join).str.lstrip()        

In [95]:
from gensim.models import Word2Vec

In [96]:
listforw2v=list(RNS[RNS['NameFilter']!=''].\
                    drop_duplicates(subset=['NameFilter'])['NameFilter'].\
                    apply(nltk.word_tokenize))

In [98]:
model = Word2Vec(listforw2v, min_count=1)

In [99]:
import itertools

def word2vecscore(topwords,mod):
    words2score=list(itertools.combinations(topwords,2))
    scores=[]
    for i in words2score:
        score=mod.wv.similarity(i[0],i[1])
        scores.append(score)
    
    coherence=np.median(scores)
    return coherence   


In [101]:
model.wv.similarity('cell','cells')

0.18516116